# 다음 셀레니움으로 뉴스 데이터 수집

검색어 : 종목코드 상한가

옵션 : 날짜, 오래된순

문영식의 무적의 크롤링 코드 완성!

In [ ]:
# 스크롤 내리는 기능 입니다.
# actions = driver.find_element(By.CSS_SELECTOR, 'body') 
# for i in range(0): #스크롤 내릴 횟수 설정
#     actions.send_keys(Keys.END)
#     time.sleep(2)

# 크롬을 통해 크롤링 하기 위해 웹 드라이버 import
from selenium import webdriver

# 구글 드라이버 자동설치
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# 요소 찾을 때 사용하는 By 모듈
from selenium.webdriver.common.by import By

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

# 페이지 렌더링 될 때까지 기다리게 하려고 쓰는 time 함수
import time
import random
t1 = random.uniform(1,3) # 1~3초 사이
t2 = random.uniform(3,5) # 2~5초 사이
t3 = random.uniform(5,8) # 5~8초 사이

# 브라우저 컨트롤 통신 관련하여 설정하려고 쓰는 모듈
import ssl

# 데이터 프레임 만들려고
import pandas as pd

# 브라우저 컨트롤할때 반영할 옵션들
# options = Options()
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
# 봇 탐지 방어 코드..
options.add_argument("--disable-blink-features=AutomationControlled")
# 크롤링 창 사이즈 고정
options.add_argument("window-size=1920x1080")
# 원래 셀레니움은 창이 뜨는 상태로 크롤링됨, 하지만 headless 옵션을 추가하면 창이 안떠도 암묵적으로 크롤링이 된다.
# options.add_argument("headless")

# 창 크기 최대화
# driver.maximize_window()

# ssl 인증서 검증을 비활성화, 가끔 셀레니움으로 켯을 때 사이트 자체에서 ssl 인증을 허가 하지 않는 접속을 막아버린다.
ssl._create_default_https_context = ssl._create_unverified_context

# 브라우저를 컨트롤 할 드라이버 객체를 선언, 원래는 크롬드라이버가 설치되어야 하지만, 아래 문구로 엄청 쉽게 사용할 수 있다.
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 페이지 3초동안 로드될때까지 기다림, 안켜지면 오류 발생시킴 암묵적 wait
driver.implicitly_wait(10)

""" # 주소창 접속
driver.get("https://www.daum.net/")
time.sleep(t1)

# 종목 검색
driver.find_element(By.XPATH, '//*[@id="q"]').send_keys(keyword)
time.sleep(t1)

# 검색 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="daumSearch"]/fieldset/div/div/button[3]').click()
time.sleep(t1)

# 뉴스탭 들어가기
driver.find_element(By.XPATH, '//*[@id="daumGnb"]/div[1]/ul/li[2]/a').click()
time.sleep(t1)
"""

# 상한가 CSV 파일 읽기
sanghanga_df = pd.read_csv('./datas/final_sanghanga_df.csv')

# 종목 셋팅
# keyword = input('검색할 종목명을 입력하세요: ')
# goodday = input('상한가 도달한 날짜를 입력하세요: YYYYMMNN')
names = sanghanga_df['종목명']
sanghanga_df['종목코드'] = sanghanga_df['종목코드'].apply(lambda x: '{:06d}'.format(x)) # 자릿수 유지
keywords = sanghanga_df['종목코드']
gooddays = sanghanga_df['날짜']
labels = sanghanga_df['라벨'] # 상한가 날 이후 10일 동안 최대 20% 상승 했는지 아닌지 [0: 추가 상승 없음, 1: 추가 상승 있음]


for name, keyword, goodday, realUp in zip(names, keywords, gooddays, labels): # 데이터셋에서 종목명 기준으로 반복.
    # df에 담을 정보
    stocks = [] # 종목 명
    titles = [] # 뉴스 제목
    hrefs = [] # 뉴스 주소
    contexts = [] # 뉴스 본문
    realUps = [] # 종목 진짜 올랐는지.

    keyword = str(keyword)
    # 주식 종목 조건에 맞게 검색 [종목코드, 상한가 날짜]
    url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+'%s'%keyword+' 상한가'+'&p=1&sort=old'+'&sd='+'%s'%goodday+'000000'+'&ed='+'%s'%goodday+'235959'+'&period=u'
    driver.get(url)
    time.sleep(t3)

    # 종목 뉴스 검색했을 때의 URL을 나중에 되돌아올 수 있도록 따로 보관해 두기.
    bowl_url = driver.current_url

    #뉴스 title, href 추출
    elements = driver.find_elements(By.CSS_SELECTOR, "#dnsColl > div:nth-child(1) > ul > li")
    if len(elements) != 0 : # 뉴스가 없는 경우도 있음. -> 현재 종목명이 변경돼서 없던 것임.. 종목코드로 검색하면 됨. 일단 그래도 냅둠
        for element in elements:
            element = element.find_element(By.CSS_SELECTOR, "div.item-title")
            element = element.find_element(By.TAG_NAME, "a")
            realUps.append(realUp)
            stocks.append(name) # 주식명
            titles.append(element.text) #뉴스제목
            hrefs.append(element.get_attribute("href")) #뉴스링크
            
        # 뉴스 링크에 접속해서 본문 따기
        for i in range(0,len(hrefs)):
            driver.get(hrefs[i])
            time.sleep(t2)
            try : 
                context = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/div[2]/section').text
            except : # 본문 구조가 다른 경우가 있음
                context = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/div/section').text
            context = context.replace("\n", " ")
            contexts.append(context)
            time.sleep(t1)

        # 데이터프레임 생성
        df = pd.DataFrame({
            'realUp': realUps,
            'stock': stocks,
            'title': titles,
            'href': hrefs,
            'context': contexts
        })

        # CSV 파일로 저장
        df.to_csv('./datas/news.csv', mode='a' ,header=False, index=False)
        
        time.sleep(t3)
    
print('뉴스 데이터 수집이 완료되었습니다.')

In [42]:
import os
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('./datas/news.csv', usecols=[0,1,2,3,4])

# 갯수 세기
df.index


RangeIndex(start=0, stop=13670, step=1)

In [ ]:

# 뉴스 내용 최대 길이 확인
df['context'].str.len().max()

In [47]:
# CSV 파일 읽기
df = pd.read_csv('./datas/news.csv', usecols=[0,1,4])
df.head()

,realUp,stock,context
0,1,차바이오텍,[서울경제] 차바이오텍(085660)은 2일 오전 10시 23분 현재 19.33% ...
1,1,차바이오텍,[서울경제] 오후 12시 14분 현재 안트로젠(065660)이 +30.00% 오른 ...
2,1,차바이오텍,[서울경제] 오후 1시 12분 현재 차바이오텍(085660)이 +30.00% 오른 ...
3,1,차바이오텍,[서울경제] 2일 오후 1시 30분 현재 코스닥은 전일 대비 10.72p(+1.34...
4,1,차바이오텍,[서울경제] 2일 오후 2시 0분 현재 코스닥은 전일 대비 11.39p(+1.43%...


In [ ]:
import pandas as pd
df = pd.read_csv('./datas/real_final_sanghanga_df.csv', usecols=[1,3,17])
df.head()

,날짜,종목코드,라벨
0,20180102,85660,1
1,20180102,900100,1
2,20180102,41190,1
3,20180103,263810,0
4,20180103,56730,0


In [ ]:
len(set(df['종목코드'].tolist()))

1445